### <font color='blue'> Looking at DAC files 

* This notebook is just for trying out dataloading  
* encoded with DAC (off line) using this command:
```
python3 -m dac encode  /scratch/syntex/PisWinAppBee_long_44/audio-train  --output /scratch/syntex/PisWinAppBee_long_44/dac-train --n_quantizers 4 --device "cuda"  --model_type "44khz" --model_tag "latest" --args.save /scratch/syntex/PisWinAppBee_long_44/dac_params.txt
```
</font>

In [ ]:
DEVICE='cpu' # either 'cuda' or 'cpu'

%pwd
%cd /app

import dac
from audiotools import AudioSignal

import torch
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as ipd
import math

# and for creating a custom dataset and loader:
from torch.utils.data import Dataset, DataLoader
import os
import dac

from audiotools import AudioSignal

In [ ]:
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())

if DEVICE == 'cuda' :
    print(f'memeory on cuda 0 is  {torch.cuda.get_device_properties(0).total_memory/1e9}')
    #print(f'memeory on cuda 1 is  { torch.cuda.get_device_properties(1).total_memory/1e9}')
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

device = torch.device(DEVICE) # if the docker was started with --gpus all, then can choose here.
#device = torch.device('cpu') # if the docker was started with --gpus all, then can choose here
device

In [ ]:
model_path = dac.utils.download(model_type="44khz") 
model = dac.DAC.load(model_path)

model.to(device); #wanna see the model? remove the semicolon
model.eval();  # need to be "in eval mode" in order to set the number of quantizers

In [ ]:
print(f'./{dataroot}/foo')

<hr>

In [ ]:
import shutil
import os

def setup_directory_and_copy_file(dest_dir, src_file):
    shutil.rmtree(dest_dir, ignore_errors=True)  # Remove the directory if it exists
    os.makedirs(dest_dir, exist_ok=True)  # Create the directory
    shutil.copy(src_file, dest_dir)  # Copy the file

dacdir= dataroot+'/foo'
fpath=dataroot + "/44kHz/N4/PisWinAppBee_sparse_dac/DSPistons--rate_exp-00.50.dac"

# Example usage
setup_directory_and_copy_file(dacdir, fpath)



In [ ]:
!ls ./{dataroot}/foo

<hr>

In [ ]:
with torch.no_grad():    
    dacfile = dac.DACFile.load(fpath)
    # FIRST - Decompress it back to an AudioSignal\ from codes to z (1024) to signal   
    print(f'dacfile.codes shape is: {dacfile.codes.shape}')

    # SECOND - Decompress directly using model.decompress   
    yAudioSig = model.decompress(dacfile)  #an AudioSignal



In [ ]:
dacfile

In [ ]:
# AudioSignal docs: https://descriptinc.github.io/audiotools/readme.html
yAudioSig.widget()

yAudioSig.audio_data.shape	

# xsig=xTensor.cpu().detach().numpy()[0,0,:]
# plt.plot(xsig)
# ipd.Audio(xsig, rate=44100)

In [ ]:
220500/44100

### <font color='blue'> Creating a custom dataset 
</font>

In [ ]:
class CustomDACDataset(Dataset):
    def __init__(self, data_dir):
        """
        Args:
            data_dir (string): Directory with all the data files.
        """
        self.data_dir = data_dir
        self.file_names = os.listdir(data_dir)

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        fpath = os.path.join(self.data_dir, self.file_names[idx])
        dacfile=dac.DACFile.load(fpath)  # Load the data file
        data = dacfile.codes

        # Assuming data is a tensor of shape [1, N, T]
        # We remove the first dimension to get a tensor of shape [N, T]
        data = data.squeeze(0)

        # The input is the data itself
        input_data = data[:, :-1]  # All time steps except the last one
        # The target is the data shifted by one time step
        target_data = data[:, 1:]  # All time steps except the first one

        # Transpose the last dimensions so we get [T, N] for the transformer
        return input_data.transpose(0, 1), target_data.transpose(0, 1)


In [ ]:
# Define the directory where the data files are stored
data_dir = dacdir

# Create an instance of the dataset
dataset = CustomDACDataset(data_dir=data_dir)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example of iterating over the DataLoader
for batch_idx, (inputs, targets) in enumerate(dataloader):
    pass
    # Your training code here
    # inputs: batch of input data of shape [batch_size, N, T-1]
    # targets: corresponding batch of target data of shape [batch_size, N, T-1]
    
    #print(f"Batch {batch_idx + 1}")
    #print(f"Inputs shape: {inputs.shape}")
    #print(f"Targets shape: {targets.shape}")
print(f"Batch {batch_idx + 1}")